In [ ]:
## Install all dependencies
!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')
# Download 'punkt_tab' data package
nltk.download('punkt_tab')
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:

path='/content/Jobs_NYC_Postings.csv'

In [ ]:
# Load data, handling potential errors
# The 'error_bad_lines' argument is deprecated in newer versions of pandas.
# Use 'on_bad_lines' instead for similar functionality.
try:
    df = pd.read_csv(path, quoting=3, on_bad_lines='skip')  # Skip bad lines
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")
    # Handle the error appropriately, e.g., try to fix the CSV or skip it
    df = pd.DataFrame()  # Create an empty DataFrame if reading fails

# Check data
df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
689000,OFFICE OF THE COMPTROLLER,External,1,Legal Intern - Law and Adjustment,SUMMER GRADUATE INTERN,Non-Competitive-5,10232,00,Legal Affairs,P,Student,19,21,Hourly,"""1 Centre St.","N.Y.""",Law and Adjustment,"""NOTE: All applicants must be current matriculated students at an accredited United States law school. The New York City Comptroller",an independently elected official,is the Chief Financial Officer of the City of New York. The mission of the office is to promote the financial integrity and operational effectiveness of New York City government,in order to secure a thriving future for all New Yorkers and build a more just,equitable,and resilient city. We utilize the tools and responsibilities that the City Charter assigns us to ensure that City government is budgeting wisely,investing strategically,living up to its promises and obligations,and keeping a sharp eye on the long-term challenges to come. Authorized by the New York City Charter,the Comptroller,through the Bureau of Law and Adjustment (BLA),negotiates and approves all monetary settlement of claims and lawsuits involving the City of New York. BLA investigates and,when in the best interest of the city,settles pre-litigation claims for and against the City of New York. In resolving claims before litigation is commenced,BLA protects the fiscal interest of the City while ensuring that pre-litigation claims are resolved in a fair and just manner,and in accordance with applicable laws. BLA furthers this goal in its review,evaluation,and authorization of requests to settle litig...,rooting out fraud,and identifying trends,as well as,coordinating with the Law Department and othe...,1 Centre St. in Manhattan/New York. - All int...,Monday â Thursday,28 hours per week. - The employment period wi...,"2025.""",...,Suffolk,Putnam,Westchester,Rockland,or Orange County. To determine if the residen...,please discuss with the agency representative...,11/19/2024,01-MAR-2025,11/19/2024,01/14/2025
635466,DEPARTMENT OF TRANSPORTATION,Internal,1,BOB- Assistant Civil Engineer,ASSISTANT CIVIL ENGINEER,Competitive-1,20210,00,"""Engineering",Architecture,"& Planning""",F,Experienced (non-manager),62370,93587,Annual,55 Water St Ny Ny,East River & Movable,"""Civil Service Title- Assistant Civil Engineer * IN ORDER TO BE CONSIDERED FOR THIS POSITION CANDIDATE MUST BE SERVING PERMANENTLY IN THE TITLE OF ASSISTANT CIVIL ENGINEER",OR REACHABLE ON EXAM #5015. PLEASE ATTACH YOUR PROOF IF FILING / NOTICE OF RESULTS. The Assistant Civil Engineer will perform work of moderate difficulty and responsibility. Engage in research,investigations or studies related to engineering functions and activities. They will develop drawings,write specifications and prepare estimates. They will participate in inspection operations by observing,checking and certifying the installation of material an equipment. Perform other related duties and will be assigned to various Bridge projects. Skills- At least one to three years of Bridge related experience. Proficient in Microsoft word/excel. A Valid New York State Driver's License. Work Location- 55 Water St,NY,"NY""",1. A baccalaureate degree from an accredited college in civil engineering and one year of full-time satisfactory experience in civil engineering work; or 2. A baccalaureate degree from an accredited college and a Master's degree from an accredited college in civil engineering.,At least one to three years of Bridge related experience. Proficient in Microsoft word/excel. A Valid New York State Driver's License.,"""The City of New York is an inclusive equal opportunity employer committed to recruiting and retaining a diverse workforce and p

In [ ]:
# Show column name
df.columns

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Classification', 'Title Code No', 'Level',
       'Job Category', 'Full-Time/Part-Time indicator', 'Career Level',
       'Salary Range From', 'Salary Range To', 'Salary Frequency',
       'Work Location', 'Division/Work Unit', 'Job Description',
       'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')

In [ ]:
df =df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']]
df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Business Title,Job Description,Minimum Qual Requirements,Preferred Skills
689000,OFFICE OF THE COMPTROLLER,External,1,Legal Intern - Law and Adjustment,SUMMER GRADUATE INTERN,Non-Competitive-5,10232,00,Legal Affairs,P,Student,19,21,Hourly,"""1 Centre St.","N.Y.""",Law and Adjustment,"""NOTE: All applicants must be current matriculated students at an accredited United States law school. The New York City Comptroller",an independently elected official,is the Chief Financial Officer of the City of New York. The mission of the office is to promote the financial integrity and operational effectiveness of New York City government,in order to secure a thriving future for all New Yorkers and build a more just,equitable,and resilient city. We utilize the tools and responsibilities that the City Charter assigns us to ensure that City government is budgeting wisely,investing strategically,living up to its promises and obligations,and keeping a sharp eye on the long-term challenges to come. Authorized by the New York City Charter,the Comptroller,through the Bureau of Law and Adjustment (BLA),negotiates and approves all monetary settlement of claims and lawsuits involving the City of New York. BLA investigates and,when in the best interest of the city,settles pre-litigation claims for and against the City of New York. In resolving claims before litigation is commenced,BLA protects the fiscal interest of the City while ensuring that pre-litigation claims are resolved in a fair and just manner,and in accordance with applicable laws. BLA furthers this goal in its review,as well as,NaN,"""New York City residency is generally required...",City Employees in certain titles who have wor...
635466,DEPARTMENT OF TRANSPORTATION,Internal,1,BOB- Assistant Civil Engineer,ASSISTANT CIVIL ENGINEER,Competitive-1,20210,00,"""Engineering",Architecture,"& Planning""",F,Experienced (non-manager),62370,93587,Annual,55 Water St Ny Ny,East River & Movable,"""Civil Service Title- Assistant Civil Engineer * IN ORDER TO BE CONSIDERED FOR THIS POSITION CANDIDATE MUST BE SERVING PERMANENTLY IN THE TITLE OF ASSISTANT CIVIL ENGINEER",OR REACHABLE ON EXAM #5015. PLEASE ATTACH YOUR PROOF IF FILING / NOTICE OF RESULTS. The Assistant Civil Engineer will perform work of moderate difficulty and responsibility. Engage in research,investigations or studies related to engineering functions and activities. They will develop drawings,write specifications and prepare estimates. They will participate in inspection operations by observing,checking and certifying the installation of material an equipment. Perform other related duties and will be assigned to various Bridge projects. Skills- At least one to three years of Bridge related experience. Proficient in Microsoft word/excel. A Valid New York State Driver's License. Work Location- 55 Water St,NY,"NY""",1. A baccalaureate degree from an accredited college in civil engineering and one year of full-time satisfactory experience in civil engineering work; or 2. A baccalaureate degree from an accredited college and a Master's degree from an accredited college in civil engineering.,At least one to three years of Bridge related experience. Proficient in Microsoft word/excel. A Valid New York State Driver's License.,"""The City of New York is an inclusive equal opportunity employer committed to recruiting and retaining a diverse workforce and providing a work environment that is free from discrimination and harassment based upon any legally protected status or protected characteristic",including but not limited to an individual's sex,race,color,ethnicity,national origin,veteran status,NaN,New York City Residency is not required for th...,05/25/2024
687673,DEPT OF DESIGN & CONSTRUCTION,External,1,Project Manager,ASSISTANT ARCHITECT,Competitive-1,21210,00,"""Engineering",Architecture,"& Planning""",F,Experienced (non-manager),64242,73878,Annual,30-30 Thomson Ave L I City Qns,PUB BLDGS/EXEC,"""Hours: Full-T

In [ ]:
# Create a new column called 'data' and merge the values of the other columns into it
df['data'] = df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
df.drop(['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills'], axis=1, inplace=True)
# Preview the updated dataframe
print(df.head())

In [ ]:
# Tag data
data = list(df['data'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

In [ ]:
# Model initialization
model = Doc2Vec(vector_size = 50,
min_count = 5,
epochs = 100,
alpha = 0.001
)

In [ ]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

291


In [ ]:
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/100


Training epoch 2/100


Training epoch 3/100


Training epoch 4/100


Training epoch 5/100


...

Training epoch 96/100


Training epoch 97/100


Training epoch 98/100


Training epoch 99/100


Training epoch 100/100
Model saved


In [ ]:
pdf = PyPDF2.PdfReader('/content/Resume.pdf')
resume = ""
for i in range(len(pdf.pages)):
    pageObj = pdf.pages[i]
    resume += pageObj.extract_text()

In [ ]:

# JD by input text:
jd = input("Paste your JD here: ")

Paste your JD here: We are seeking an experienced Oracle PL/SQL Developer to join our dynamic team in Toronto, ON. As a key member of our organization, you will play a critical role in developing, maintaining, and optimizing database solutions for large-scale applications in finance, insurance, and data analytics. The ideal candidate will have a proven track record of working with Oracle databases, creating efficient PL/SQL scripts, and managing data in compliance with industry standards. This is a full-time position that offers an opportunity to collaborate with cross-functional teams to implement innovative database solutions that meet complex business requirements.  In this role, you will be responsible for designing, testing, and maintaining PL/SQL stored procedures, functions, packages, and triggers to support business applications. You will optimize database queries using advanced techniques such as bulk collections, cursor management, and table partitioning to ensure efficient d

In [ ]:
jd = """
We are seeking an experienced Oracle PL/SQL Developer to join our dynamic team in Toronto, ON. As a key member of our organization, you will play a critical role in developing, maintaining, and optimizing database solutions for large-scale applications in finance, insurance, and data analytics. The ideal candidate will have a proven track record of working with Oracle databases, creating efficient PL/SQL scripts, and managing data in compliance with industry standards. This is a full-time position that offers an opportunity to collaborate with cross-functional teams to implement innovative database solutions that meet complex business requirements.

In this role, you will be responsible for designing, testing, and maintaining PL/SQL stored procedures, functions, packages, and triggers to support business applications. You will optimize database queries using advanced techniques such as bulk collections, cursor management, and table partitioning to ensure efficient data handling. Additional responsibilities include performing data migrations and ETL processes using tools like SQL*Loader, Data Pump, and external tables. You will analyze and resolve performance bottlenecks using query optimization techniques and tools such as AWR and ADDM while ensuring compliance with data protection standards like GDPR. Your contributions will also involve collaborating with global teams to gather requirements, document technical designs, and implement solutions that align with business goals.

The successful candidate will have at least six years of experience as an Oracle PL/SQL Developer, with expertise in Oracle 11g/12c, tools such as SQL Developer and TOAD, and a strong knowledge of data modeling and normalization techniques. You should have hands-on experience with ETL processes, partitioned tables, bitmap indexes, and collections (nested tables, VARRAYs, associative arrays). Familiarity with Agile methodologies, tools like JIRA, and DevOps practices such as Git and Jenkins is essential. Excellent problem-solving, debugging, and performance-tuning skills are critical for success in this role.

While not required, experience in the financial or insurance sectors, exposure to front-end technologies (HTML, PHP, JavaScript), and knowledge of data warehousing tools such as Informatica would be an asset. Additionally, certifications like Oracle Certified Associate (OCA) in PL/SQL and advanced database certifications will strengthen your candidacy.

In return, we offer a competitive salary, performance-based bonuses, and a comprehensive benefits package that includes health, dental, and retirement plans. You will have access to professional development opportunities, certification reimbursements, and flexible work schedules with hybrid/remote options. As part of a collaborative and innovative team, you will work on impactful projects that challenge your expertise and contribute to the success of global clients.

Join us and take your career to the next level as we deliver cutting-edge database solutions and empower businesses to thrive in a fast-paced digital world."""

In [ ]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()

    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)

    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = ' '.join(text.split())

    return text

In [ ]:
# Apply to CV and JD
input_CV = preprocess_text(resume)
input_JD = preprocess_text(jd)

In [ ]:
# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

In [ ]:
# Visualization
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = similarity,
    mode = "gauge+number",
    title = {'text': "Matching percentage (%)"},
    #delta = {'reference': 100},
    gauge = {
        'axis': {'range': [0, 100]},
        'steps' : [
            {'range': [0, 50], 'color': "#FFB6C1"},
            {'range': [50, 70], 'color': "#FFFFE0"},
            {'range': [70, 100], 'color': "#90EE90"}
        ],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()

# Print notification
if similarity < 50:
    print(colored("Low chance, need to modify your CV!", "red", attrs=["bold"]))
elif similarity >= 50 and similarity < 70:
    print(colored("Good chance but you can improve further!", "yellow", attrs=["bold"]))
else:
    print(colored("Excellent! You can submit your CV.", "green", attrs=["bold"]))